In [1]:
import torch
import numpy as np
import pandas as pd
import os
import sys

In [184]:
from sklearn.metrics import classification_report

In [23]:
from torch.utils.data import TensorDataset, DataLoader

In [27]:
import torch.nn as nn
from torch import optim

In [2]:
from sklearn.model_selection import train_test_split

In [69]:
from sklearn.preprocessing import StandardScaler

### Baseline Split

In [99]:
data = pd.read_csv('../data/gesture-recognition-and-biometrics-electromyogram-grabmyo-1.0.2/features_split.csv')

In [100]:
data.head()

,Unnamed: 0,session,participant,gesture,index,iemg,mav,ssi,myopulse,wflen,...,kurtosis_f_w3,kurtosis_f_w4,kurtosis_f_w5,kurtosis_f_w6,kurtosis_f_w7,kurtosis_f_w8,kurtosis_f_w9,kurtosis_f_w10,kurtosis_f_w11,kurtosis_f_w12
0,0,1,1,10,0,11996.863087,0.041842,997.135823,6.224512,8143.485828,...,61.940683,116.130710,31.371157,107.176549,64.459302,241.232060,54.880235,56.951175,42.644289,118.918727
1,1,1,1,10,0,10550.150337,0.036796,1019.627142,6.443555,6646.721181,...,160.171639,87.566832,154.616093,280.595181,160.756112,98.589493,135.655396,194.396727,294.858908,537.374741
2,2,1,1,10,0,9230.366582,0.032193,630.013233,6.033789,6023.438546,...,195.666850,90.568034,94.299202,276.105575,101.698579,49.607516,150.348389,151.177847,156.860637,115.905382
3,3,1,1,10,0,15949.504704,0.055627,1995.234560,4.999609,9318.559036,...,33.886272,80.377706,46.373047,64.445165,31.711430,33.328942,50.491362,81.876212,69.070232,142.723324
4,4,1,1,10,0,15936.956936,0.055584,1830.412922,5.870117,10476.465579,...,48.630658,39.722917,25.079065,84.877336,29.229362,48.843301,44.917769,39.354396,78.705208,176.820713


In [101]:
data.columns

Index(['Unnamed: 0', 'session', 'participant', 'gesture', 'index', 'iemg',
       'mav', 'ssi', 'myopulse', 'wflen',
       ...
       'kurtosis_f_w3', 'kurtosis_f_w4', 'kurtosis_f_w5', 'kurtosis_f_w6',
       'kurtosis_f_w7', 'kurtosis_f_w8', 'kurtosis_f_w9', 'kurtosis_f_w10',
       'kurtosis_f_w11', 'kurtosis_f_w12'],
      dtype='object', length=573)

In [102]:
data.drop(columns=["Unnamed: 0"], inplace=True)

In [103]:
data

,session,participant,gesture,index,iemg,mav,ssi,myopulse,wflen,diffvar,...,kurtosis_f_w3,kurtosis_f_w4,kurtosis_f_w5,kurtosis_f_w6,kurtosis_f_w7,kurtosis_f_w8,kurtosis_f_w9,kurtosis_f_w10,kurtosis_f_w11,kurtosis_f_w12
0,1,1,10,0,11996.863087,0.041842,997.135823,6.224512,8143.485828,0.001888,...,61.940683,116.130710,31.371157,107.176549,64.459302,241.232060,54.880235,56.951175,42.644289,118.918727
1,1,1,10,0,10550.150337,0.036796,1019.627142,6.443555,6646.721181,0.001438,...,160.171639,87.566832,154.616093,280.595181,160.756112,98.589493,135.655396,194.396727,294.858908,537.374741
2,1,1,10,0,9230.366582,0.032193,630.013233,6.033789,6023.438546,0.001169,...,195.666850,90.568034,94.299202,276.105575,101.698579,49.607516,150.348389,151.177847,156.860637,115.905382
3,1,1,10,0,15949.504704,0.055627,1995.234560,4.999609,9318.559036,0.003030,...,33.886272,80.377706,46.373047,64.445165,31.711430,33.328942,50.491362,81.876212,69.070232,142.723324
4,1,1,10,0,15936.956936,0.055584,1830.412922,5.870117,10476.465579,0.003688,...,48.630658,39.722917,25.079065,84.877336,29.229362,48.843301,44.917769,39.354396,78.705208,176.820713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15346,3,9,9,0,7435.557234,0.025933,369.818221,6.280762,5792.222983,0.001070,...,29.589425,25.460694,182.962449,34.957161,18.846525,37.714119,26.569683,32.614845,12.793219,16.657922
15347,3,9,9,0,10471.625678,0.036522,713.052200,6.967187,8628.380136,0.002070,...,18.226247,35.715660,113.822913,64.223311,8.214120,39.211218,29.910432,26.426457,24.892159,82.241098
15348,3,9,9,0,11279.138159,0.039339,815.440185,6.917773,9229.795138,0.002355,...,15.486525,29.417342,25.153035,19.632020,15.590478,15.462391,34.940941,37.564280,14.208021,15.811267
15349,3,9,9,0,9210.946156,0.032125,563.422838,7.359570,8040.357766,0.001839,...,14.527208,20.281059,20.418717,18.471741,14.276295,10.083487,27.337412,12.804675,20.220452,37.494222


In [104]:
features = data.iloc[:, 4:]

In [105]:
features

,iemg,mav,ssi,myopulse,wflen,diffvar,dasd,willison,power_f1,power_f2,...,kurtosis_f_w3,kurtosis_f_w4,kurtosis_f_w5,kurtosis_f_w6,kurtosis_f_w7,kurtosis_f_w8,kurtosis_f_w9,kurtosis_f_w10,kurtosis_f_w11,kurtosis_f_w12
0,11996.863087,0.041842,997.135823,6.224512,8143.485828,0.001888,0.034480,10277,0.002297,0.004143,...,61.940683,116.130710,31.371157,107.176549,64.459302,241.232060,54.880235,56.951175,42.644289,118.918727
1,10550.150337,0.036796,1019.627142,6.443555,6646.721181,0.001438,0.030668,6420,0.003276,0.005005,...,160.171639,87.566832,154.616093,280.595181,160.756112,98.589493,135.655396,194.396727,294.858908,537.374741
2,9230.366582,0.032193,630.013233,6.033789,6023.438546,0.001169,0.027090,5052,0.001570,0.002424,...,195.666850,90.568034,94.299202,276.105575,101.698579,49.607516,150.348389,151.177847,156.860637,115.905382
3,15949.504704,0.055627,1995.234560,4.999609,9318.559036,0.003030,0.045251,19492,0.004655,0.006518,...,33.886272,80.377706,46.373047,64.445165,31.711430,33.328942,50.491362,81.876212,69.070232,142.723324
4,15936.956936,0.055584,1830.412922,5.870117,10476.465579,0.003688,0.047223,21471,0.003665,0.005650,...,48.630658,39.722917,25.079065,84.877336,29.229362,48.843301,44.917769,39.354396,78.705208,176.820713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15346,7435.557234,0.025933,369.818221,6.280762,5792.222983,0.001070,0.026210,5307,0.001265,0.000794,...,29.589425,25.460694,182.962449,34.957161,18.846525,37.714119,26.569683,32.614845,12.793219,16.657922
15347,10471.625678,0.036522,713.052200,6.967187,8628.380136,0.002070,0.035779,11624,0.002672,0.002797,...,18.226247,35.715660,113.822913,64.223311,8.214120,39.211218,29.910432,26.426457,24.892159,82.241098
15348,11279.138159,0.039339,815.440185,6.917773,9229.795138,0.002355,0.038498,14370,0.002550,0.002242,...,15.486525,29.417342,25.153035,19.632020,15.590478,15.462391,34.940941,37.564280,14.208021,15.811267
15349,9210.946156,0.032125,563.422838,7.359570,8040.357766,0.001839,0.033699,10798,0.001746,0.001153,...,14.527208,20.281059,20.418717,18.471741,14.276295,10.083487,27.337412,12.804675,20.220452,37.494222


In [106]:
labels = data.iloc[:, 2] - 1

In [107]:
labels.unique()


array([ 9, 10, 11, 12, 13, 14, 15, 16,  0,  1,  2,  3,  4,  5,  6,  7,  8],
      dtype=int64)

In [108]:
labels

0        9
1        9
2        9
3        9
4        9
        ..
15346    8
15347    8
15348    8
15349    8
15350    8
Name: gesture, Length: 15351, dtype: int64

In [109]:
features

,iemg,mav,ssi,myopulse,wflen,diffvar,dasd,willison,power_f1,power_f2,...,kurtosis_f_w3,kurtosis_f_w4,kurtosis_f_w5,kurtosis_f_w6,kurtosis_f_w7,kurtosis_f_w8,kurtosis_f_w9,kurtosis_f_w10,kurtosis_f_w11,kurtosis_f_w12
0,11996.863087,0.041842,997.135823,6.224512,8143.485828,0.001888,0.034480,10277,0.002297,0.004143,...,61.940683,116.130710,31.371157,107.176549,64.459302,241.232060,54.880235,56.951175,42.644289,118.918727
1,10550.150337,0.036796,1019.627142,6.443555,6646.721181,0.001438,0.030668,6420,0.003276,0.005005,...,160.171639,87.566832,154.616093,280.595181,160.756112,98.589493,135.655396,194.396727,294.858908,537.374741
2,9230.366582,0.032193,630.013233,6.033789,6023.438546,0.001169,0.027090,5052,0.001570,0.002424,...,195.666850,90.568034,94.299202,276.105575,101.698579,49.607516,150.348389,151.177847,156.860637,115.905382
3,15949.504704,0.055627,1995.234560,4.999609,9318.559036,0.003030,0.045251,19492,0.004655,0.006518,...,33.886272,80.377706,46.373047,64.445165,31.711430,33.328942,50.491362,81.876212,69.070232,142.723324
4,15936.956936,0.055584,1830.412922,5.870117,10476.465579,0.003688,0.047223,21471,0.003665,0.005650,...,48.630658,39.722917,25.079065,84.877336,29.229362,48.843301,44.917769,39.354396,78.705208,176.820713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15346,7435.557234,0.025933,369.818221,6.280762,5792.222983,0.001070,0.026210,5307,0.001265,0.000794,...,29.589425,25.460694,182.962449,34.957161,18.846525,37.714119,26.569683,32.614845,12.793219,16.657922
15347,10471.625678,0.036522,713.052200,6.967187,8628.380136,0.002070,0.035779,11624,0.002672,0.002797,...,18.226247,35.715660,113.822913,64.223311,8.214120,39.211218,29.910432,26.426457,24.892159,82.241098
15348,11279.138159,0.039339,815.440185,6.917773,9229.795138,0.002355,0.038498,14370,0.002550,0.002242,...,15.486525,29.417342,25.153035,19.632020,15.590478,15.462391,34.940941,37.564280,14.208021,15.811267
15349,9210.946156,0.032125,563.422838,7.359570,8040.357766,0.001839,0.033699,10798,0.001746,0.001153,...,14.527208,20.281059,20.418717,18.471741,14.276295,10.083487,27.337412,12.804675,20.220452,37.494222


In [115]:
scaler = StandardScaler()

X = scaler.fit_transform(features)
Y = labels.values

In [116]:
Y.shape

(15351,)

In [117]:
X.shape

(15351, 568)

In [118]:
X

array([[ 0.02392332,  0.02392332, -0.1874717 , ..., -0.04878718,
        -0.11424713,  0.38753344],
       [-0.18785798, -0.18785798, -0.17863791, ...,  1.18393595,
         2.47507057,  3.60160943],
       [-0.38105841, -0.38105841, -0.3316644 , ...,  0.796314  ,
         1.05833522,  0.36438855],
       ...,
       [-0.08114296, -0.08114296, -0.25883529, ..., -0.22266456,
        -0.40618315, -0.40441409],
       [-0.38390132, -0.38390132, -0.35781874, ..., -0.44472878,
        -0.34445757, -0.2378717 ],
       [-0.33314875, -0.33314875, -0.33456336, ..., -0.3679266 ,
        -0.42503833, -0.2247458 ]])

In [185]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=43)

In [186]:
x_train

array([[-0.11629552, -0.11629552, -0.19289876, ..., -0.19079338,
        -0.21617383, -0.23099812],
       [ 3.45405924,  3.45405924,  3.97590219, ..., -0.3774615 ,
        -0.37834971, -0.37972356],
       [ 0.19801118,  0.19801118, -0.11286955, ..., -0.28453885,
        -0.32990027, -0.21279221],
       ...,
       [ 0.09244361,  0.09244361, -0.10603224, ...,  0.8706434 ,
        -0.03801353,  0.09171287],
       [-0.19932551, -0.19932551, -0.29204992, ..., -0.3768488 ,
        -0.17473026, -0.38905995],
       [-0.30814251, -0.30814251, -0.30755708, ..., -0.01050093,
        -0.23205439, -0.20590404]])

In [187]:
x_test

array([[-0.27462652, -0.27462652, -0.3221948 , ..., -0.45198375,
        -0.31674569, -0.41387085],
       [-0.1133239 , -0.1133239 , -0.24361123, ...,  0.32942267,
        -0.21838578, -0.27355456],
       [ 2.71427156,  2.71427156,  2.38994258, ..., -0.15937449,
         0.22752784,  0.39048761],
       ...,
       [ 0.83047173,  0.83047173,  0.28591516, ...,  0.01952809,
        -0.26177294, -0.23809514],
       [-0.18717363, -0.18717363, -0.28588523, ..., -0.31540792,
        -0.35467747, -0.33105603],
       [-0.13234052, -0.13234052, -0.21027761, ...,  0.64867224,
        -0.18352401,  0.04790248]])

In [188]:
# x_train = x_train.values
# x_test = x_test.values

# y_train = y_train.values
# y_test = y_test.values

In [189]:
y_test


array([3, 1, 6, ..., 6, 2, 3], dtype=int64)

In [206]:
def simple_model(input_shape, output_shape):
    model = nn.Sequential(
        nn.Linear(input_shape, 32),  # Input layer: Fully connected (linear) with 64 units
        nn.ReLU(),  # Activation function: ReLU
        nn.Linear(32, 64),
        nn.ReLU(),  # Activation function: ReLU
        nn.Linear(64, output_shape)  # Output layer: Fully connected (linear) with 'output_shape' units
    )
    
    return model

In [207]:
train_dataset = TensorDataset(torch.tensor(x_train).type(torch.float32), torch.tensor(y_train).type(torch.LongTensor))
test_dataset = TensorDataset(torch.tensor(x_test).type(torch.float32), torch.tensor(y_test).type(torch.LongTensor))

In [208]:
len(np.unique(y_train))

17

In [220]:
# Define batch size and whether to shuffle the data
batch_size = 128
shuffle = True

# Create data loaders for training and testing
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [221]:
len(train_dataset)

12280

In [222]:
x_train.shape

(12280, 568)

In [223]:
model = simple_model(x_train.shape[1], len(np.unique(y_train)))

In [224]:
criterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [225]:
# Define training parameters
num_epochs = 50

In [226]:
# Training loop
for epoch in range(num_epochs):
    # Training phase
    model.train()
    total_train_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    # Validation phase
    model.eval()
    total_val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(test_loader):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate and log metrics
    train_loss = total_train_loss / len(train_loader)
    val_loss = total_val_loss / len(test_loader)
    val_accuracy = 100 * correct / total

    print(f"Epoch [{epoch + 1}/{num_epochs}] - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

Epoch [1/50] - Train Loss: 1.4938, Val Loss: 1.0819, Val Accuracy: 63.79%
Epoch [2/50] - Train Loss: 0.8986, Val Loss: 0.9366, Val Accuracy: 69.16%
Epoch [3/50] - Train Loss: 0.7881, Val Loss: 0.8759, Val Accuracy: 72.45%
Epoch [4/50] - Train Loss: 0.7137, Val Loss: 0.8795, Val Accuracy: 73.66%
Epoch [5/50] - Train Loss: 0.6595, Val Loss: 0.8652, Val Accuracy: 74.31%
Epoch [6/50] - Train Loss: 0.6511, Val Loss: 0.8050, Val Accuracy: 75.87%
Epoch [7/50] - Train Loss: 0.6010, Val Loss: 0.9471, Val Accuracy: 73.10%
Epoch [8/50] - Train Loss: 0.6138, Val Loss: 0.8277, Val Accuracy: 75.55%
Epoch [9/50] - Train Loss: 0.5269, Val Loss: 0.8251, Val Accuracy: 77.53%
Epoch [10/50] - Train Loss: 0.4757, Val Loss: 0.8799, Val Accuracy: 77.89%
Epoch [11/50] - Train Loss: 0.4662, Val Loss: 0.8168, Val Accuracy: 78.35%
Epoch [12/50] - Train Loss: 0.4685, Val Loss: 1.0161, Val Accuracy: 75.71%
Epoch [13/50] - Train Loss: 0.4553, Val Loss: 0.9261, Val Accuracy: 77.21%
Epoch [14/50] - Train Loss: 0.4248

In [231]:
def predict(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    y_pred = []

    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(test_loader):
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            y_pred.extend(predicted.tolist())

    return y_pred

In [232]:
y_test_pred = predict(model, test_loader)

In [233]:
y_test_pred

[3,
 1,
 6,
 1,
 2,
 14,
 6,
 4,
 11,
 8,
 7,
 15,
 16,
 1,
 7,
 14,
 15,
 9,
 16,
 2,
 2,
 4,
 8,
 2,
 10,
 13,
 0,
 3,
 5,
 4,
 11,
 5,
 7,
 5,
 11,
 8,
 13,
 4,
 9,
 16,
 6,
 15,
 4,
 7,
 16,
 15,
 0,
 9,
 5,
 15,
 9,
 7,
 2,
 14,
 3,
 9,
 6,
 2,
 3,
 8,
 7,
 11,
 10,
 4,
 1,
 13,
 3,
 12,
 9,
 6,
 9,
 6,
 4,
 13,
 5,
 16,
 9,
 10,
 7,
 4,
 16,
 13,
 11,
 10,
 6,
 1,
 4,
 1,
 13,
 9,
 7,
 6,
 7,
 14,
 12,
 1,
 6,
 6,
 2,
 3,
 16,
 15,
 15,
 8,
 5,
 5,
 4,
 3,
 7,
 3,
 8,
 8,
 8,
 2,
 3,
 16,
 5,
 16,
 4,
 8,
 3,
 15,
 13,
 7,
 13,
 10,
 8,
 7,
 9,
 2,
 10,
 12,
 6,
 9,
 8,
 9,
 15,
 6,
 12,
 0,
 11,
 13,
 7,
 15,
 11,
 14,
 12,
 1,
 12,
 8,
 2,
 8,
 12,
 6,
 13,
 0,
 8,
 10,
 10,
 16,
 14,
 15,
 3,
 14,
 10,
 8,
 15,
 13,
 8,
 14,
 5,
 4,
 0,
 16,
 0,
 13,
 9,
 0,
 15,
 7,
 5,
 1,
 7,
 11,
 7,
 11,
 2,
 15,
 1,
 1,
 2,
 16,
 5,
 4,
 5,
 15,
 14,
 1,
 2,
 12,
 16,
 5,
 10,
 10,
 11,
 8,
 2,
 7,
 7,
 9,
 15,
 1,
 12,
 1,
 11,
 6,
 14,
 16,
 10,
 0,
 14,
 12,
 10,
 6,
 11,
 1,
 10,
 1,

In [234]:
report = classification_report(y_test, y_test_pred)
print(report)

              precision    recall  f1-score   support

           0       0.63      0.66      0.65       159
           1       0.66      0.68      0.67       165
           2       0.72      0.78      0.75       180
           3       0.76      0.66      0.70       204
           4       0.85      0.82      0.84       186
           5       0.91      0.87      0.89       180
           6       0.68      0.73      0.70       184
           7       0.81      0.85      0.83       170
           8       0.85      0.70      0.77       192
           9       0.78      0.84      0.81       207
          10       0.99      0.95      0.97       167
          11       0.94      0.90      0.92       191
          12       0.90      0.91      0.91       149
          13       0.89      0.90      0.89       192
          14       0.88      0.91      0.89       177
          15       0.73      0.71      0.72       191
          16       0.90      0.97      0.93       177

    accuracy              